## 1. 文本特征抽取 
    这里部分特征是基于词的，我们基于字识别，可参考ner iobes体系，将词特征转为字特征

### 1). 词性特征
    将词性信息转化为特征-> S-w:单独的词 B-w:词的开头 I-w:词的中间 E-w:词的结尾
    例如：武汉分公司与

    假设词性:武汉/ns
            分公司/n
            近日/t
            与/p
            
    特征为:
            武	B-ns
            汉	E-ns
            分	B-n
            公	I-n
            司 	E-n
            近	B-t
            日	E-t
            与	S-p
### 2). 其他特征
     其他，如字形、笔画、拼音、专用字典等特征。或用正则表达式发现模式，符合模式的抽取为特征。

In [1]:
import jieba
import jieba.posseg as pseg

jieba.initialize()

import warnings
warnings.filterwarnings('ignore')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/96/0jnny1gj02j8pgfvb9by368m0000gn/T/jieba.cache
Loading model cost 0.624 seconds.
Prefix dict has been built successfully.


In [2]:
sentence = "中国电信武汉分公司近日与公安、交管部门展开深入合作"

In [3]:
print('词', '\t|', '词性')
print('————————————————')
for (word, flag) in pseg.cut(sentence):
    print(word, '\t|', flag)

词 	| 词性
————————————————
中国电信 	| nt
武汉 	| ns
分公司 	| n
近日 	| t
与 	| p
公安 	| n
、 	| x
交管部门 	| n
展开 	| v
深入 	| v
合作 	| vn


In [8]:
def get_word_posseg_feature(sentence):
    posseg_list = []
    for (word, flag) in pseg.cut(sentence):
        if len(word) == 1:
            posseg_list.append('S-' + flag)
        else:
            posseg_list.append('B-' + flag)
            for _ in word[1:-1]:
                posseg_list.append('I-' + flag)                
            posseg_list.append('E-' + flag)
    return posseg_list

posseg_list = get_word_posseg_feature(sentence)

KeyboardInterrupt: 

In [9]:
for index, w in enumerate(sentence):
    print('\t'.join([w, posseg_list[index]]))

中	B-nt
国	I-nt
电	I-nt
信	E-nt
武	B-ns
汉	E-ns
分	B-n
公	I-n
司	E-n
近	B-t
日	E-t
与	S-p
公	B-n
安	E-n
、	S-x
交	B-n
管	I-n
部	I-n
门	E-n
展	B-v
开	E-v
深	B-v
入	E-v
合	B-vn
作	E-vn


### 2. json to bio
#### 为使用机器学习方法进行NER，将json对象的标注文件转化为bio标签
        中	B-company
        国	I-company
        电	I-company
        信	I-company
        武	I-company
        汉	I-company
        分	I-company
        公	I-company
        司	I-company
        近	O
        日	O
        与	O
        公	B-government
        安	I-government
        、	O
        交	B-government
        管	I-government
        部	I-government
        门	I-government
        展	O
        开	O
        深	O
        入	O
        合	O
        作	O
####  将json文件中所有句子转为bio形式，各个句子之间空行隔开

In [10]:
import os
import json
import codecs

import util

In [11]:
bio_data_folder = './bio'

In [8]:
train_path = './clue/train.json'
test_path = './clue/test.json'

train_samples = util.read_json(train_path)
test_samples = util.read_json(test_path)

print(f'训练集大小:\t{len(train_samples)}')
print(f'测试集大小:\t{len(test_samples)}')

训练集大小:	10748
测试集大小:	1343


In [9]:
# 将一条样本转化为bio序列
def sample_to_bio(sample):
    text = sample['text']
    label = sample['label']
    # 初始化所有样本的bio序列为O
    tag_list = ['O'] * len(text)

    for tag, tag_items in label.items():
        # 获取所有实体标签
        for tag_text, tag_index in tag_items.items():
            beg, end = tag_index[0]
            end += 1
            # 做文本内容校验
            assert text[beg: end] == tag_text, (text[beg: end], tag_text)
        # 如果实体长度为1，那么只有1个B标签
        if end - beg == 1:
            tag_list[beg] = 'B-' + tag
        else:   # 实体长度大于1，则开头为B，后面全为I标签
            tag_list[beg] = 'B-' + tag
            for i in range(beg + 1, end):  
                tag_list[i] = 'I-' + tag
    # 获取pos tag特征
    posseg_list = get_word_posseg_feature(text)
    # 将BIO标签序列 按行拼接
    bio_text = '\n'.join([' '.join([text[index], posseg_list[index], tag_list[index]]) 
                            for index in range(len(text))])
    return bio_text


def samples_to_bios(samples):
    # 转化所有的样本为bio标签，每个样本即每个句子用空行隔开
    bios = '\n\n'.join([sample_to_bio(sample) for sample in samples])
    return bios

test_bios = samples_to_bios(test_samples)
print(test_bios[:190])

In [11]:
# 提取训练集与测试集的bio序列
for json_file in [train_path, test_path]:
    samples = util.read_json(json_file)
    bios = samples_to_bios(samples)
    bios_file = os.path.join(bio_data_folder, os.path.basename(json_file)[:-4] + 'bio')
    with codecs.open(bios_file, 'w', encoding='utf-8') as f:
        f.write(bios)

In [12]:
!ls ./bio

test.bio  train.bio
